# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096950


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:38,  3.53s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:09,  2.58s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:49,  1.89s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:34,  1.37s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:13<00:01,  3.21it/s]

Rendering models:  90%|█████████ | 27/30 [00:14<00:00,  3.58it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  4.22it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  4.25it/s]

not-logged-in-14800bc0cb17aef85ae6    0.003016
not-logged-in-3b0b929f4475fd276561    0.003950
not-logged-in-3b0b929f4475fd276561    0.090840
not-logged-in-0687f3923f81ac7bd40a    0.002350
Alexander00                           0.000245
jnarayanbvg                           0.001802
Sophmoss                              0.000122
awright5                              0.000555
Nagol929                              0.000838
thiagoxxxx                            0.000148
theetick                              0.004565
sn346808                              0.000299
ElisabethB                            0.000115
ToriPizza                             0.001323
not-logged-in-d69b7704236a6d85afe5    0.000569
LeeJoseph                             0.013328
not-logged-in-623cb7cd00f5dbcaf5ac    0.000065
djkv                                  0.001171
ovincent                              0.002993
MaximeRoyer1993                       0.000106
not-logged-in-02fb9245b87202c863de    0.000458
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())